# Imports

In [0]:
!pip install https://files.pythonhosted.org/packages/aa/53/858c95f2994356e237d1fa4008f2675c871dbdd1a25cb52029c5c4ea1c85/synthetic_datasets-0.1.11-py3-none-any.whl
import keras
import numpy as np
import matplotlib.pyplot as plt
import random, math
from matplotlib import animation, rc
from IPython.display import HTML


Using TensorFlow backend.


# Boring methods

In [0]:
%matplotlib inline

BATCH_SIZE = 12

plt.rcParams['image.cmap'] = 'viridis'

def plot_dataset(data):
    fig, ax = plt.subplots()

    def init():
        ax.cla()
        return ()

    def animate(i):
        ax.imshow(data[i%len(data)])
        return ()

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=BATCH_SIZE, interval=700, blit=True)
    plt.close(fig)
    fig.set_size_inches(12, 12, True)
    return HTML(anim.to_jshtml())

  
def add_circle(data, xpos, ypos, radius, darkness, width):
    NUM_CIRCLE_DOTS = 600
    TWO_PI = 6.2838
    DIM=64
    for i in range(NUM_CIRCLE_DOTS):
        for r in range(int(radius), int(radius)+width):
            rad = TWO_PI * i/NUM_CIRCLE_DOTS
            x = int(round(r*math.cos(rad)+xpos))
            y = int(round(-r*math.sin(rad)+ypos))
            if x >= 0 and x < DIM and y >= 0 and y < DIM:
                data[x,y] = data[x,y] - darkness


# Synthetic Data

In [0]:
# Synthetic Dataset - open source - benchmarking new aspects of architecture
from synthetic_datasets import NoiseCircle

generator = next(NoiseCircle(batch_size=BATCH_SIZE, width=4))

# Visualizing the NoiseCircle synthetic dataset

In [0]:

batch, _ = next(generator)
plot_dataset(batch)
# As you can see, none are full circles and only 64x64. So we have to train this neural net to identify circles

# Keras Functional API - Creating a neural network with 3 subnets

#

In [0]:
from keras.models import Model
from keras.layers import Input, Reshape, SeparableConv2D, Conv2D, MaxPooling2D, Dense, Flatten, AveragePooling2D, UpSampling2D, BatchNormalization, Lambda
from keras.optimizers import Adam
from keras.layers.merge import add

F  = 48      # The number of filters in each convolutional layer
N  = 48      # Used to size the dense layers
KS = (7,7)   # The kernel size
ACT = 'elu'
LR  = 0.0001
INIT = 'glorot_normal'
PAD = 'same'


# Specify an input later and specify the shape of it
i = a = Input(shape=(64,64))


a = Reshape((64, 64, 1)) (a)

# 6 convolutional layers with batch normalization - normalizes data so easier to process
a = Conv2D(F, KS, activation=ACT, kernel_initializer=INIT, padding=PAD) (a)
a = Conv2D(F, KS, activation=ACT, kernel_initializer=INIT, padding=PAD) (a)
a = Conv2D(F, KS, activation=ACT, kernel_initializer=INIT, padding=PAD) (a)
a = Conv2D(F, KS, activation=ACT, kernel_initializer=INIT, padding=PAD) (a)
a = AveragePooling2D()(a)

a = Conv2D(F, KS, activation=ACT, kernel_initializer=INIT, padding=PAD) (a)
a = AveragePooling2D()(a)

a = Conv2D(F, KS, activation=ACT, kernel_initializer=INIT, padding=PAD) (a)
a = BatchNormalization()(a)

trunk = a

# 3 different inferences: x, y, and radius coordinates that we want the model to learn
# A neural net with 3 subset branches

# Subnet to predict the X coordinate

# Flatten then send into dense layers and bring it down into one final dense layer
# Kernal: weight of a layer

x = Lambda(lambda q: q[:,:,:,:16])(trunk)
x = Flatten() (x)
x = Dense(N//2, activation=ACT, kernel_initializer=INIT) (x)
x = Dense(N,    activation=ACT, kernel_initializer=INIT) (x)
x = Dense(1,          name="X", kernel_initializer=INIT) (x)


# Subnet to predict the Y coordinate
y = Lambda(lambda q: q[:,:,:,16:32])(trunk)
y = Flatten() (y)
y = Dense(N//2, activation=ACT, kernel_initializer=INIT) (y)
y = Dense(N,    activation=ACT, kernel_initializer=INIT) (y)
y = Dense(1,          name="Y", kernel_initializer=INIT) (y)


# Subnet to predict the Radius
r = Lambda(lambda q: q[:,:,:,32:48])(trunk)
r = Flatten() (r)
r = Dense(N//2, activation=ACT, kernel_initializer=INIT) (r)
r = Dense(N,    activation=ACT, kernel_initializer=INIT) (r)
r = Dense(1,          name="R", kernel_initializer=INIT) (r)


# Combine all layers into model
model = Model(inputs = i, outputs = [x, y, r])
model.compile(optimizer=Adam(LR, beta_1=0.5, beta_2=0.99), loss='mse')


# Provides complete description of model
print(model.summary())

model.fit_generator(generator, steps_per_epoch=20, epochs=50)




__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64)       0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 64, 64, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 48)   2400        reshape_1[0][0]                  
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 48)   112944      conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

In [0]:



combined = []
for k in range(25):
  batch, _ = next(generator)
  p = model.predict(batch)
  for i, sample in enumerate(batch):
      add_circle(sample, p[0][i][0], p[1][i][0], p[2][i][0], -.5, 4)  # The prediction will appear bright
      combined.append(sample)


plot_dataset(np.array(combined))